In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

%matplotlib inline

In [2]:
inp_df = pd.read_csv(r'TimeStamp_clients.csv')

Preprocessing data - deleted unnecessary columns to reduce noise


In [3]:
inp_df = inp_df.drop(['Suffix'], axis = 1)
inp_df = inp_df.drop(['Zip'], axis = 1)
inp_df = inp_df.drop(['Birthdate'], axis = 1)
inp_df = inp_df.drop(['Gender'], axis = 1)
inp_df = inp_df.drop(['Race'], axis = 1)
inp_df = inp_df.drop(['Creatingagency'], axis = 1)
inp_df = inp_df.drop(['Visitdt'], axis = 1)
inp_df = inp_df.drop(['Visittm'], axis = 1)
inp_df["Fullname"] = inp_df["Firstname"] +' '+inp_df["Lastname"]
inp_df = inp_df.drop(['Lastname'], axis = 1)
inp_df = inp_df.drop(['Firstname'], axis = 1)
temp = inp_df['Scandatetime'].str.split(' ')
inp_df['Scandate'] = temp.str.get(0)
inp_df = inp_df.drop(['Scandatetime'], axis = 1)
inp_df = inp_df[inp_df['Entrydate'].notnull()]
inp_df['Entrydate'] = pd.to_datetime(inp_df['Entrydate'])
inp_df['Scandate'] = pd.to_datetime(inp_df['Scandate'])
inp_df['Difference'] = inp_df.groupby(['Fullname'])['Scandate'].apply(lambda x: x.diff())
inp_df['Year'] = inp_df['Scandate'].dt.to_period('Q')


In [4]:
inp_df.head(6)

,Entrydate,Fullname,Scandate,Difference,Year
0,2017-12-20,Ivan Castro,2018-01-02,NaT,2018Q1
1,2017-10-23,Carlos Ramos,2018-01-02,NaT,2018Q1
2,2014-04-12,Richard Bennett,2018-01-02,NaT,2018Q1
3,2017-06-25,Francis Lewis,2018-01-02,NaT,2018Q1
4,2017-09-03,Daniel Deramus,2018-01-02,NaT,2018Q1
5,2017-05-25,Rena Davis,2018-01-02,NaT,2018Q1


In [5]:
inp_df[inp_df['Fullname'] == "Ivan Castro"]

,Entrydate,Fullname,Scandate,Difference,Year
0,2017-12-20,Ivan Castro,2018-01-02,NaT,2018Q1
5537,2017-12-20,Ivan Castro,2018-02-01,30 days,2018Q1
10486,2017-12-20,Ivan Castro,2018-02-28,27 days,2018Q1
24317,2017-12-20,Ivan Castro,2018-05-10,71 days,2018Q2
40761,2017-12-20,Ivan Castro,2018-07-28,79 days,2018Q3
42126,2017-12-20,Ivan Castro,2018-08-04,7 days,2018Q3
49714,2017-12-20,Ivan Castro,2018-09-13,40 days,2018Q3
56057,2017-12-20,Ivan Castro,2018-10-15,32 days,2018Q4
75169,2017-12-20,Ivan Castro,2019-01-23,100 days,2019Q1
82235,2017-12-20,Ivan Castro,2019-02-27,35 days,2019Q1


In [6]:
inp2_df = inp_df.groupby(['Fullname', 'Year']).size().reset_index(name='Size')


In [7]:
inp2_df = inp2_df.pivot(index = 'Fullname', columns = 'Year')
inp2_df.columns = inp2_df.columns.droplevel(0)
inp2_df = inp2_df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('.'))
inp2_df["Total"] = inp2_df["2018Q1"] + inp2_df["2018Q2"] + inp2_df["2018Q3"] + inp2_df["2018Q4"] + inp2_df["2019Q1"] + inp2_df["2019Q2"] + inp2_df["2019Q3"]

In [8]:
inp2_df.head()
#inp2_df.shape

Year,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,Total
Fullname,,,,,,,,
Melissa Roberts,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0
Timmon Wyane Hall,4.0,1.0,4.0,2.0,5.0,1.0,0.0,17.0
Vanessa Polanco,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
douglas swab,1.0,1.0,0.0,0.0,1.0,2.0,0.0,5.0
ivonna molina,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [9]:
inp2_df.describe()

Year,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,Total
count,11851.000000,11851.000000,11851.000000,11851.000000,11851.000000,11851.000000,11851.000000,11851.000000
mean,1.351785,1.591596,1.530504,1.489410,1.575226,1.376846,0.657328,9.572694
std,3.234054,3.700602,3.700812,3.626013,3.784888,3.305040,1.638138,19.410994
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000
max,58.000000,58.000000,58.000000,65.000000,66.000000,55.000000,27.000000,315.000000


In [10]:
inp2_df = inp2_df.sort_values("Total",ascending=False)

In [11]:
inp2_df.head(10)

Year,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,Total
Fullname,,,,,,,,
Christine Stamas,58.0,58.0,41.0,43.0,43.0,46.0,26.0,315.0
Hung Dang,24.0,31.0,53.0,46.0,43.0,33.0,23.0,253.0
Geoffrey Musazi,28.0,39.0,44.0,38.0,58.0,27.0,18.0,252.0
Vera Menchan,31.0,32.0,41.0,39.0,49.0,45.0,14.0,251.0
Sheila & Marco Alvarez,21.0,43.0,58.0,33.0,31.0,36.0,21.0,243.0
Willie Barber,21.0,21.0,42.0,34.0,46.0,53.0,23.0,240.0
Santos Morales,0.0,4.0,38.0,51.0,66.0,53.0,27.0,239.0
Riliere Louis-Jean,31.0,46.0,41.0,47.0,41.0,21.0,11.0,238.0
Andrea Belcher,1.0,0.0,37.0,65.0,63.0,53.0,12.0,231.0
